In [1]:
import transformers as ppb
import pandas as pd
import torch
import numpy as np
from tqdm import tqdm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from scipy import spatial

In [2]:
device

device(type='cuda')

In [3]:
corpus_dic = {}
with open('data/corpus_coding_test.tsv', 'r', encoding='utf8') as lines:
    for line in lines:
        idx, text = line.split('\t')
        temp_list = []
        temp_list.append(text)
        corpus_dic[idx] = temp_list

In [4]:
corpus_dic['90041']

['"I hear it all the time boo! Too bad I ain\'t got the patience for anyone else\'s problems!"  That got me laughing even through all the pain. It really wasn\'t that good. I hugged me best friend. I\'m pretty sure I squeezed the life out of her.   I hung up hastily. Another day, another phone call he ignores.  "Cleo go and answer the door!!" I swear I feel like a bloody slave in this house!  I barely had enough time to go upstairs and change when a knock came at the door.  I got to work quickly cleaning the kitchen, bathroom and living room, which wasn\'t easy since this woman was in the way! When I finished I had sweat pouring all down my face. I was shattered!  Her face immediately darkened and I knew this wasn\'t good. "Ehhhh! If you\'re not a baby anymore then you can go and find a job and start paying rent! Now hurry up and tidy! Uncle Tony will be here in 30 minutes!!!" She turned her attention back to the tv, where she was watching some poorly edited  film on Nollywood  "Mum!! 

In [5]:
model = ppb.DistilBertModel
model = model.from_pretrained('distilbert-base-uncased')
tokenizer = ppb.DistilBertTokenizer
tokenizer = tokenizer.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
model = model.to(device)
model.eval()
for k, value in tqdm(corpus_dic.items()):
    tokenizerd = tokenizer.encode(value[0], add_special_tokens=True, truncation=True)
    input_ids = torch.tensor(np.array(tokenizerd)).type(torch.LongTensor).to(device)
    input_ids = input_ids.unsqueeze(dim=0)
    #print(input_ids.shape)
    with torch.no_grad():
        last_hidden_states = model(input_ids)
    vector = last_hidden_states[0][:, 0, :].cpu().numpy()
    value.append(vector)
    

100%|██████████| 999/999 [00:47<00:00, 21.17it/s]


In [7]:
corpus_dic['90041'][1]

array([[-6.91091642e-03, -3.05959079e-02, -5.22356592e-02,
        -4.98602986e-01, -1.20080993e-01, -8.10839087e-02,
         3.39835346e-01,  1.76513687e-01, -1.95727929e-01,
        -2.33761042e-01,  7.50518218e-02, -1.64736994e-03,
        -1.71018288e-01,  3.77302229e-01, -4.62734476e-02,
         8.83444324e-02,  1.24823399e-01,  4.81824726e-01,
         8.36847350e-02,  1.52414188e-01,  2.10798070e-01,
        -1.91835269e-01,  2.18405813e-01,  1.20383888e-01,
        -8.67653638e-02, -5.60752526e-02,  3.93622890e-02,
        -7.41535723e-02,  2.33038768e-01, -1.65317744e-01,
         2.25213438e-01,  2.69308444e-02,  9.19919237e-02,
        -2.47886121e-01,  1.20865993e-01, -1.96079358e-01,
        -8.39167535e-02, -1.39299527e-01, -2.73773760e-01,
         1.11212090e-01, -1.36757150e-01,  1.86503995e-02,
         4.41099778e-02, -6.35304973e-02, -1.27451599e-01,
        -6.93896562e-02, -3.36073327e+00, -1.75650164e-01,
        -2.36813445e-02, -2.16080412e-01,  3.00795823e-0

In [8]:
for key, value in tqdm(corpus_dic.items()):
    scores = {}
    for k in corpus_dic.keys():
        if key != k:
            dist = spatial.distance.cosine(value[1], corpus_dic[k][1])
            scores[k] = dist
    value.append(scores)

100%|██████████| 999/999 [00:36<00:00, 27.15it/s]


In [9]:
corpus_dic['90041'][2]

{'48027': 0.14771950244903564,
 '44081': 0.04509389400482178,
 '54461': 0.0303652286529541,
 '37461': 0.04487574100494385,
 '74610': 0.05833995342254639,
 '56339': 0.04250067472457886,
 '37417': 0.08036541938781738,
 '95706': 0.04503333568572998,
 '48424': 0.04889911413192749,
 '70694': 0.06302523612976074,
 '65036': 0.0618290901184082,
 '47559': 0.075092613697052,
 '19503': 0.049542367458343506,
 '11702': 0.058359384536743164,
 '18721': 0.06794548034667969,
 '50155': 0.03855639696121216,
 '44890': 0.08535349369049072,
 '45637': 0.10803991556167603,
 '70596': 0.05187416076660156,
 '35142': 0.06567263603210449,
 '64559': 0.07992756366729736,
 '68913': 0.08019590377807617,
 '59837': 0.037740468978881836,
 '50146': 0.06334066390991211,
 '67032': 0.042320430278778076,
 '44506': 0.06957316398620605,
 '60410': 0.059655845165252686,
 '17882': 0.03232920169830322,
 '77314': 0.13240337371826172,
 '43676': 0.047122418880462646,
 '33534': 0.07229971885681152,
 '33190': 0.057291269302368164,
 '327

In [10]:
for key, value in tqdm(corpus_dic.items()):
    sorted_dict = {}
    sorted_keys = sorted(value[2], key=value[2].get)
    value.append(sorted_keys[:5])

100%|██████████| 999/999 [00:00<00:00, 4287.51it/s]


In [11]:
corpus_dic['90041'][3]

['39263', '53338', '08591', '52673', '40523']

In [12]:
corpus_dic['72169'][0]

'I was cut off by a tap on my shoulder. I bent my head backwards to see Aimee. I grinned. No feelings! I stopped mid grin and got up.   Crap. What if she got realesed? What if she escaped? There goes my ticket to life out of this hell hole. I knew she\'d leave. She probably left cause of me. Cra-   I sat down in the familiar chairs waiting for Aimme. I glanced at the wall clock that hung above the small table. She\'s 10 minutes late.  I sat up ready to see Aimee. I grinned thinking of her face when i kissed her. She\'ll fall for me in no time. Might as well have fun before I crush her wittle heart.   Jason\'s POV    I offered her my chair. So much for no feelings and rudeness. I quickly sat on the chair and pulled her on my lap. Back to the plan. I groaned. Just when I thought Allie was gonna do something right, she pulls one on me. And ruins my day. I sighed and walked over to my lump, falling asleep.   "Hell no! Your lonely! Embrace any company you get!" she said with a wink.   "Are 

In [13]:
corpus_dic[corpus_dic['72169'][3][0]][0]

'Shit. "Erm, sure. Okay." I answered nervously after a couple of seconds contemplation. We are turned to each other, and got ready. Dan placed his hands on my waist, and mine around his neck. Why? No. No, this can\'t be happening. I\'d fetched Dan\'s coffee and my juice and was going back to the table. Just before I turned the corner I peered round, hoping Dan had his back to me so I could make him jump. (Brooke\'s POV) ******************* "Shhh, don\'t worry Dan, she\'ll understand if you explain everything to her. She\'ll understand don\'t worry." She hugged me close and rubbed circles into my back. "What am I going to tell Brooke?" I said as tears began to fall down my face. "Jesus fucking Christ! What the fuck was that?!" I screamed, as the security guards carried her away. Leasha stared open mouthed at the scene. I sat down on a stool and Leasha ran over and hugged me. "I know you like her. A lot. But she can\'t kiss for shit. Here, let me show you how it\'s done." I had to proces

In [14]:
output = ''
for key, value in tqdm(corpus_dic.items()):
    output += key
    for id_doc in value[3]:
        output += '\t'+id_doc+':'+str(round(value[2][id_doc], 3))
    output += '\n'
output = output[:-1]

100%|██████████| 999/999 [00:00<00:00, 34451.34it/s]


In [15]:
print(output)

90041	39263:0.021	53338:0.022	08591:0.026	52673:0.027	40523:0.028
48027	94139:0.055	08363:0.058	33534:0.064	35203:0.074	92261:0.074
44081	41467:0.024	06580:0.024	33067:0.025	58706:0.026	42620:0.028
54461	03434:0.023	92438:0.024	72974:0.025	32859:0.025	65030:0.026
37461	95706:0.023	73343:0.028	63502:0.028	32602:0.029	45094:0.03
74610	33690:0.036	92743:0.039	34805:0.043	37253:0.043	39423:0.044
56339	06580:0.025	80868:0.025	50428:0.027	55317:0.028	35046:0.029
37417	84695:0.034	53150:0.039	52089:0.039	52133:0.04	34997:0.041
95706	70596:0.021	91121:0.021	90609:0.021	17565:0.022	37461:0.023
48424	77951:0.017	50498:0.021	62215:0.022	38894:0.023	11914:0.024
70694	64559:0.028	49903:0.03	56924:0.031	37871:0.032	37170:0.034
65036	51022:0.021	61193:0.022	06580:0.022	46783:0.027	96846:0.027
47559	38226:0.04	33190:0.041	01660:0.044	39310:0.047	34879:0.047
19503	36200:0.024	44704:0.027	41427:0.027	51713:0.032	00093:0.033
11702	49589:0.028	72169:0.033	37017:0.034	48424:0.034	33032:0.034
18721	35892:0.

In [16]:
with open('data/output.tsv', 'w') as txt_file:
    txt_file.write(output)